In [1]:
# MOSTLY PROTOTYPING AND INITAL DATA EXPLORATION, very dirty

from glob import glob
import os
import pydicom as dicom
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from scipy.ndimage import center_of_mass

In [3]:
data_folder = Path('manifest-1686081801328')  # Set your data folder here
# Print metadata
metadata_df = pd.read_csv(data_folder/'metadata.csv')
metadata_df.tail()

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
1198,2.25.179810891732019793459496497832233425115.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1724,2.25.96089192699936339821029186870284643178,NCI PDMR Tumor Characterization,06-11-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.47 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1724\06-1...,2023-11-17T16:10:23.58
1199,1.3.6.1.4.1.5962.1.2.0.1670154536.70508.0.175.2,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1728,1.3.6.1.4.1.5962.1.2.0.1670154536.70508.0.175.1,NCI PDMR Tumor Characterization,05-20-2020,PDM Mouse Overview,PixelMed,SR,Acquisition Context SR Storage,1.2.840.10008.5.1.4.1.1.88.71,1,4.41 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1728\05-2...,2023-11-17T16:10:24.266
1200,2.25.308200546840067706797055294221268540073.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1728,2.25.295695138875747345906456588608783888488,NCI PDMR Tumor Characterization,06-11-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.46 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1728\06-1...,2023-11-17T16:10:27.462
1201,2.25.308200546840067706797055294221268540073,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1728,2.25.295695138875747345906456588608783888488,NCI PDMR Tumor Characterization,06-11-2020,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,36,22.51 MB,.\PDMR-Texture-Analysis\BL0382-F1232-1728\06-1...,2023-11-17T16:10:45.81
1202,2.25.243475922033648146678409629073405401370,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,997537-175-T-1327,2.25.244508863228859243918875004939444840754,NCI PDMR Tumor Characterization,11-14-2018,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,36,22.51 MB,.\PDMR-Texture-Analysis\997537-175-T-1327\11-1...,2023-11-17T16:12:46.909


In [ ]:
metadata_df[metadata_df['Subject ID'] == 'BL0382-F1232-1714']

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
1157,1.3.6.1.4.1.5962.1.2.0.1670154536.70508.0.169.2,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,1.3.6.1.4.1.5962.1.2.0.1670154536.70508.0.169.1,NCI PDMR Tumor Characterization,05-20-2020,PDM Mouse Overview,PixelMed,SR,Acquisition Context SR Storage,1.2.840.10008.5.1.4.1.1.88.71,1,4.41 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\05-2...,2023-11-03T11:25:41.266
1159,2.25.7168843703236719329939687441569740311.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.112225220460263450892498230424741850344,NCI PDMR Tumor Characterization,08-20-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.47 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\08-2...,2023-11-03T11:25:44.002
1161,2.25.218949796884644575382271550339124402974.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.20243916927934249299121679408472823503,NCI PDMR Tumor Characterization,06-11-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.48 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\06-1...,2023-11-03T11:25:48.212
1162,2.25.7168843703236719329939687441569740311,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.112225220460263450892498230424741850344,NCI PDMR Tumor Characterization,08-20-2020,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,40,25.01 MB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\08-2...,2023-11-03T11:25:58.255
1163,2.25.218949796884644575382271550339124402974,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.20243916927934249299121679408472823503,NCI PDMR Tumor Characterization,06-11-2020,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,36,22.51 MB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\06-1...,2023-11-03T11:26:00.066
1164,2.25.257829965714272789555089379470225877285.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.274744447885883268792786353111651321308,NCI PDMR Tumor Characterization,08-06-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.48 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\08-0...,2023-11-03T11:26:00.267
1165,2.25.264074736878243749071088649598942639020.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.318107665745479241005627304927567659241,NCI PDMR Tumor Characterization,07-23-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.47 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\07-2...,2023-11-03T11:26:01.999
1166,2.25.257829965714272789555089379470225877285,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.274744447885883268792786353111651321308,NCI PDMR Tumor Characterization,08-06-2020,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,36,22.51 MB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\08-0...,2023-11-03T11:26:12.402
1167,2.25.256748391362023724633218136089904221252.1,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.96052988339886703244578515805128083072,NCI PDMR Tumor Characterization,07-08-2020,TSE45 split,Philips Medical Systems,MR,Raw Data Storage,1.2.840.10008.5.1.4.1.1.66,1,265.47 KB,.\PDMR-Texture-Analysis\BL0382-F1232-1714\07-0...,2023-11-03T11:26:14.198
1169,2.25.256748391362023724633218136089904221252,PDMR-Texture-Analysis,NO,https://doi.org/10.7937/3KQ0YK19,BL0382-F1232-1714,2.25.96052988339886703244578515805128083072,NCI PDMR Tumor Characterization,07-08-2020,TSE45 split,Philips Medical Systems,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,40

In [7]:
level1_paths = glob(str(data_folder/'PDMR-Texture-Analysis'/'*'))
data_dict = {}
for lvl1_path in level1_paths:
    level2_paths = glob(f'{lvl1_path}/*')
    lvl2_dict = {}
    for lvl2_path in level2_paths:
        lvl3_dict = {}
        lvl3_paths = glob(f'{lvl2_path}/*')
        for scan_path in lvl3_paths:
            dcm_slices = glob(f'{scan_path}/*.dcm')
            lvl3_dict[Path(scan_path).name] = dcm_slices
        lvl2_dict[Path(lvl2_path).name] = lvl3_dict
    data_dict[Path(lvl1_path).name] = lvl2_dict

In [6]:
import nrrd
data1, header1 = nrrd.read('segmentation_test\\3012 TSE45 split.nrrd')
data2, header2 = nrrd.read('segmentation_test\\Segmentation.seg.nrrd')

ModuleNotFoundError: No module named 'nrrd'

In [8]:
n_cols = 10

def plot_scan(image_paths):
    N = len(image_paths)
    n_rows = int(np.ceil(N / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 14))
    axes = axes.flatten()
    for i, img_path in enumerate(image_paths):
        ds = dicom.dcmread(img_path)
        ds.pixel_array
        axes[i].imshow(ds.pixel_array[200:-200])
        axes[i].axis('off')

    for j in range(i + 1, len(axes)):
        axes[j].axis('off') 
    return fig, axes

def plot_single(image_paths, n=17):

    ds = dicom.dcmread(image_paths[n])
    img = ds.pixel_array[200:-200]
    fig, ax = plt.subplots(figsize=(8, 12))
    ax.imshow(img)
    ax.axis('off')

    return fig, ax

lvl1_keys = list(data_dict.keys())
for key1 in lvl1_keys:
    lvl2_data = data_dict[key1]
    lvl2_keys = list(lvl2_data.keys())
    for key2 in lvl2_keys:
        lvl3_data = data_dict[key1][key2]
        lvl3_keys = list(lvl3_data.keys())
        for scan_name in lvl3_keys: # Chosing second folder as it usually contains the correct scan with multiple images
            scan_image_paths = data_dict[key1][key2][scan_name]
            for path in scan_image_paths:
                img_name = Path(path).stem
                img_order = int(img_name.split('-')[1])
                img_scan_name = scan_name
                img_inspection_name = key2
                img_inspection_date = ''.join(key2.split('-')[0:3])
                img_specimen_name = key1
                img_original_patient_id = ''.join(key2.split('-')[0:3])
                img_mouse_id = key2.split('-')[-1]
                

            # #fig, axes = plot_scan(scan_image_paths)
            # #fig.suptitle(key2) 
            # #fig.tight_layout()
            # #fig.subplots_adjust(top=0.97)
            # #fig.savefig(f'{key2}.png', transparent=True)
            # fig_single, ax = plot_single(scan_image_paths)
            # #fig_single.suptitle(key2)
            # fig_single.tight_layout()
            # fig_single.savefig(f'{key1}_{key2}_single.png', transparent=True)
        #plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [9]:
def read_dicom(path):
    d = dicom.read_file(path)
    metadata = {}
    for attr in dir(d):
        if attr[0].isupper() and attr != 'PixelData':
            try:
                metadata[attr] = getattr(d, attr)
            except AttributeError:
                pass

    metadata['InstanceNumber'] = int(metadata['InstanceNumber'])
    metadata['PixelSpacing'] = np.float32(metadata['PixelSpacing'])
    metadata['ImageOrientationPatient'] = np.float32(metadata['ImageOrientationPatient'])
    try:
        metadata['SliceLocation'] = np.float32(metadata['SliceLocation'])
    except:
        metadata['SliceLocation'] = None
    metadata['ImagePositionPatient'] = np.float32(metadata['ImagePositionPatient'])
    metadata['Rows'] = int(metadata['Rows'])
    metadata['Columns'] = int(metadata['Columns'])
    metadata['RescaleSlope'] = float(metadata['RescaleSlope'])
    metadata['RescaleIntercept'] = float(metadata['RescaleIntercept'])
    return np.array(d.pixel_array), metadata

def get_patient_data(patient_nr, experiment_nr):
    patient_data = data_dict[patient_nr]
    experiment_keys = list(patient_data.keys())
    experiment_keys_sorted = sorted(experiment_keys)
    experiment = experiment_keys_sorted[experiment_nr]

    experiment_data = data_dict[patient_nr][experiment]
    scan_keys = list(experiment_data.keys())
    scan_data = scan_keys[-1]
    slice_paths = data_dict[patient_nr][experiment][scan_data]
    if len(slice_paths) == 1:
        scan_data = scan_keys[0]
        slice_paths = data_dict[patient_nr][experiment][scan_data]

    sid2data = {}
    sid2metadata = {}
    for s in slice_paths:
        slice_id = s.split('/')[-1].split('.')[0]
        data, metadata = read_dicom(s)
        sid2data[slice_id] = data
        sid2metadata[slice_id] = metadata
    return sid2data, sid2metadata


def ct2HU(x, metadata):
    """
    Converts the raw pixel values of the CT images into Hounsfield Units (HU),
    a standardized scale used in CT scans. This scale allows for a consistent interpretation
    of radiodensity, which is crucial for identifying different types of tissues and abnormalities
    (like tumors, cysts, etc.).
    """
    x = metadata['RescaleSlope'] * x + metadata['RescaleIntercept']
    x[x < -1000] = -1000
    return x


def read_dicom_scan(patient_nr, experiment_nr):
    sid2data, sid2metadata = get_patient_data(patient_nr, experiment_nr)
    sid2position = {}
    for sid in sid2data.keys():
        sid2position[sid] = get_slice_position(sid2metadata[sid])
    sids_sorted = sorted(sid2position.items(), key=lambda x: x[1])
    sids_sorted = [s[0] for s in sids_sorted]
    z_pixel_spacing = []
    for s1, s2 in zip(sids_sorted[1:], sids_sorted[:-1]):
        z_pixel_spacing.append(sid2position[s1] - sid2position[s2])
    z_pixel_spacing = np.array(z_pixel_spacing)
    assert np.all((z_pixel_spacing - z_pixel_spacing[0]) < 0.01)

    pixel_spacing = np.array((z_pixel_spacing[0],
                              sid2metadata[sids_sorted[0]]['PixelSpacing'][0],
                              sid2metadata[sids_sorted[0]]['PixelSpacing'][1]))

    img = np.stack([ct2HU(sid2data[sid], sid2metadata[sid]) for sid in sids_sorted])

    return img, pixel_spacing

def get_slice_position(slice_metadata):
    """
    Determines the physical position of each slice in the patient's body.
    In a CT scan, slices are captured at different positions along an axis (usually the z-axis),
    and it's crucial to know the exact position of each slice for a correct 3D reconstruction of the scanned area.
    """
    orientation = tuple((o for o in slice_metadata['ImageOrientationPatient']))
    position = tuple((p for p in slice_metadata['ImagePositionPatient']))
    rowvec, colvec = orientation[:3], orientation[3:]
    normal_vector = np.cross(rowvec, colvec)
    slice_pos = np.dot(position, normal_vector)
    return slice_pos

In [10]:
PATIENT_NR = 'BL0382-F1232-1714'

patient_data = data_dict[PATIENT_NR]
time_series = []
for key_nr in range(1, len(patient_data)):
    img, pixel_spacing = read_dicom_scan(PATIENT_NR, key_nr)
    time_series.append((img, pixel_spacing))

In [11]:
from ipywidgets import interact

img = time_series[1][0] # change the first index to see slices from n-th timestamp

def explore_slices(slice_index):
    plt.imshow(img[slice_index])
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(explore_slices, slice_index=(0, len(img)-1))

interactive(children=(IntSlider(value=19, description='slice_index', max=39), Output()), _dom_classes=('widget…

<function __main__.explore_slices(slice_index)>

In [12]:
import shutil
from datetime import datetime
import os
import nibabel as nib
os.makedirs('images_2d', exist_ok=True)

def convert_dicom_to_nifti(dicom_pixels):
    # Stack the DICOM slices and convert to a NumPy array
    # Convert the image data to float and scale it
    image_data = dicom_pixels.astype(np.float32)
    image_data -= np.min(image_data)
    image_data /= np.max(image_data)

    # Create an affine matrix for the NIfTI image
    # This is a basic affine matrix, you might need to adjust it based on your DICOM metadata
    affine = np.eye(4)

    # Create the NIfTI image
    nifti_img = nib.Nifti1Image(image_data, affine)

    return nifti_img

data = {
    'img_name': [],
    'Z_order_no': [],
    'inspection_date': [],
    'scan_name': [],
    'human_patient_id': [],
    'mouse_id': [],

    'original_img_name': [],
    'inspection_name': [],

    'specimen_name': [],
}
img_id = 0
lvl1_keys = list(data_dict.keys())
print(
    lvl1_keys
)
for key1 in lvl1_keys[1:2]:
    lvl2_data = data_dict[key1]
    lvl2_keys = list(lvl2_data.keys())
    for key2 in lvl2_keys:
        lvl3_data = data_dict[key1][key2]
        lvl3_keys = list(lvl3_data.keys())
        for scan_name in lvl3_keys:
            scan_image_paths = data_dict[key1][key2][scan_name]

            if len(scan_image_paths) > 2:
                continue
            print(scan_image_paths)
            for path in scan_image_paths:
                new_img_name = f'{img_id}.nii'
                ds = dicom.dcmread(path)
                try:
                    nifti = convert_dicom_to_nifti(np.array([ds.pixel_array]))
                except Exception as e:
                    print(e)
                    continue
                nifti.to_filename(f'images_2d/{new_img_name}')
                #shutil.copy2(path, f'images_2d/{new_img_name}')
                img_name = Path(path).stem
                data['img_name'].append(new_img_name)
                data['original_img_name'].append(img_name)
                data['Z_order_no'].append(int(img_name.split('-')[1]))
                data['scan_name'].append(scan_name)
                img_original_patient_id = '-'.join(key1.split('-')[0:3])
                img_mouse_id = key1.split('-')[-1]

                data['human_patient_id'].append(img_original_patient_id)
                data['mouse_id'].append(img_mouse_id)

                date_str = '-'.join(key2.split('-')[0:3])
                data['inspection_date'].append(datetime.strptime(date_str, "%m-%d-%Y"))
                data['inspection_name'].append(key2)
                data['specimen_name'].append(key1)
                img_id += 1
df = pd.DataFrame(data)
df.to_csv('image_data_2d.csv',sep=',')



['BL0382-F1232-1714', '144126-210-T-1669', '172845-121-T-1862', '698357-238-R-1953', '698357-238-R-1955', '146476-266-R-1621', '894883-131-R-2315', '833975-119-R-1572', '695669-166-R-2043', 'BL0382-F1232-1716', '521955-158-R4-2162', '625472-104-R-1544', '765638-272-R-2017', '146476-266-R-1630', '625472-104-R-1548', '765638-272-R-2007', '625472-104-R-1545', '698357-238-R-1977', 'BL0382-F1232-1728', '894883-131-R-2320', '779769-127-R-1675', '779769-127-R-1685', '146476-266-R-1623', '287954-098-R-1995', '833975-119-R-1584', '172845-121-T-1859', '172845-121-T-1847', '172845-142-T-1265', '172845-121-T-1853', '287954-098-R-2003', '287954-098-R-2004', '287954-098-R-1987', '146476-266-R-1631', '172845-142-T-1242', '144126-210-T-1664', '466636-57-R-444', '172845-142-T-1238', '698357-238-R-1970', '172845-142-T-1246', '521955-158-R4-2177', '146476-266-R-1616', '172845-142-T-1262', '695669-166-R-2051', '172845-142-T-1247', '833975-119-R-1568', '172845-142-T-1257', '172845-121-T-1854', '833975-119-

In [15]:
def data_generator(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        yield np.load(file_path, mmap_mode='r')

data = data_generator('./data_cleaned')

X = []
y = []
for Xy in data:
    X.append((Xy[0] - Xy[0].min())/(Xy[0].max() - Xy[0].min()))
    y.append(Xy[1])


In [22]:
print(f"Cuda available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device name: {torch.cuda.get_device_name(0)}")

Cuda available: False


In [30]:
class ConvStack(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding) -> None:
        super().__init__()
        self.f = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())

    def forward(self, x):
        return self.f(x)

class UNet2D(nn.Module):
    def __init__(self, n_filters, patch_size, encoder_size=3, mid_size=4, decoder_size=3, dropout=0.25):
        super(UNet2D, self).__init__()
        self.n_filters = n_filters
        self.patch_size = patch_size
        self.encoder_size = encoder_size
        self.mid_size = mid_size
        self.decoder_size = decoder_size
        self.enc_conv1 = []
        self.encode_conv = []
        self.expand_conv1 = []
        # Convolutional layers with kernel size 3 and no padding (valid)
        # Encoder
        for i in range(encoder_size):
            if i == 0:
                self.enc_conv1.append(ConvStack(1, n_filters, kernel_size=3, padding='valid'))
            else:
                self.enc_conv1.append(ConvStack(n_filters, n_filters, kernel_size=3, padding='valid'))

        # Max pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Middle encoder layers
        for _ in range(mid_size):
            self.encode_conv.append(ConvStack(n_filters, n_filters, kernel_size=3, padding='valid'))

        # Dropout
        self.dropout1 = nn.Dropout(dropout)

        # Upscaling
        self.upscale1 = nn.ConvTranspose2d(n_filters, n_filters, kernel_size=2, stride=2)

        # Concatenation and Expansion
        for i in range(decoder_size):
            if i < 2:
                self.expand_conv1.append(ConvStack(2 * n_filters, 2 * n_filters, kernel_size=3, padding='valid'))
            elif i > 2:
                self.expand_conv1.append(ConvStack(n_filters, n_filters, kernel_size=3, padding='valid'))
            else:
                self.expand_conv1.append(ConvStack(2 * n_filters, n_filters, kernel_size=3, padding='valid'))

        # Final convolutional layer
        self.final_conv = nn.Conv2d(n_filters, 1, kernel_size=1)

    def forward(self, x):
        x1 = x
        # Encoder
        for i in range(self.encoder_size):
            x1 = self.enc_conv1[i](x1)

        x2 = self.pool1(x1)

        # Middle encoder
        for i in range(self.mid_size):
            x2 = self.encode_conv[i](x2)

        # Dropout
        x2 = self.dropout1(x2)

        # Upscale
        x3 = self.upscale1(x2)

        # Concatenation
        delta = [x1_size - x3_size for x1_size, x3_size in zip(x1.size()[2:], x3.size()[2:])]
        crop_x1 = x1[:, :, delta[0]//2:x1.size(2)-delta[0]//2, delta[1]//2:x1.size(3)-delta[1]//2]
        x3 = torch.cat((x3, crop_x1), dim=1)

        # Expansion
        for i in range(self.decoder_size):
            x3 = self.expand_conv1[i](x3)

        # Output
        x_out = torch.sigmoid(self.final_conv(x3))
        
        return x_out

    def fit(self, train_loader, num_epochs, device, verbose=True):
        optimizer = torch.optim.Adam(self.parameters())
        for epoch in range(num_epochs):
            self.train()
            for i, (images, masks) in enumerate(train_loader):
                images, masks = images.float().to(device), masks.to(device)
                optimizer.zero_grad()
                outputs = self(images)
                
                center_crop = transforms.CenterCrop((outputs.shape[-1], outputs.shape[-1]))
                resized_masks = center_crop(masks)

                loss = -1 * dice_coefficient(outputs, resized_masks).mean()
                loss.backward()
                optimizer.step()

                if verbose and i % 1 == 0:
                    print(f'Epoch : {epoch} [{i * len(images)}/{len(train_loader.dataset)} ({100. * i / len(train_loader):.0f}%)]\tLoss: {loss:.6f}')
    
    def predict(self, X, device):
        self.eval()
        predicted_scans = []
        stride = self.patch_size - 2 * self.encoder_size - 4 * self.mid_size - 2 * self.decoder_size
        compression = self.encoder_size + 2 * self.mid_size + self.decoder_size

        for image in X:
            image = torch.from_numpy(image[np.newaxis, np.newaxis, ...]).float()
            H, W = image.shape[-2], image.shape[-1]
            output_scan = torch.zeros_like(image)

            for y in range(0, H, stride):
                for x in range(0, W, stride):
                    if y + self.patch_size > H or x + self.patch_size > W:
                        continue

                    patch = image[:, :, y:y+self.patch_size, x:x+self.patch_size]
                    patch = patch.to(device)
                    output_patch = self(patch)

                    output_scan[:, :, y+compression:y+compression+stride, x+compression:x+compression+stride] = output_patch.unsqueeze(0).unsqueeze(0)

            predicted_scans.append(output_scan)

        return torch.cat(predicted_scans, dim=0)  

In [31]:
class MRIScansPatchDataset(Dataset):
    def __init__(self, images, masks, patch_size, transform=None):
        self.images = images
        self.masks = masks
        self.patch_size = patch_size
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]

        centroid = center_of_mass(mask)
        image_patch, mask_patch = self.extract_patch(image, mask, centroid)

        if self.transform:
            image_patch = self.transform(image_patch)
            mask_patch = self.transform(mask_patch)

        return image_patch[np.newaxis, ...], mask_patch[np.newaxis, ...]

    def extract_patch(self, image, mask, centroid):
        y, x = int(centroid[0]), int(centroid[1])
        half_patch = self.patch_size // 2

        # IF YOU GET THE INDEX OUT OF RANGE ERROR, THEN TRY A SMALLER PATCH SIZE!
        # patch must be a square, so in case of setting boundaries, additional padding will be required, which might reduce the quality of model
        image_patch = image[y-half_patch:y+half_patch, x-half_patch:x+half_patch]
        mask_patch = mask[y-half_patch:y+half_patch, x-half_patch:x+half_patch]

        return image_patch, mask_patch
    
def dice_coefficient(pred, target, smooth=1e-12):
    intersection = (pred * target).sum(axis=(1, 2))
    return (2. * intersection + smooth) / (pred.sum(axis=(1, 2)) + target.sum(axis=(1, 2)) + smooth)

In [32]:
from sklearn.model_selection import train_test_split

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {DEVICE}.")
BATCH_SIZE = 2
PATCH_SIZE = 80
TRAIN_EPOCHS = 10
N_FILTERS = 128

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dataset = MRIScansPatchDataset(X_train[:10],
                               y_train[:10],
                               patch_size=PATCH_SIZE)
train_loader = DataLoader(dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
model = UNet2D(n_filters=N_FILTERS,
               patch_size=PATCH_SIZE)
model.fit(train_loader,
          num_epochs=TRAIN_EPOCHS,
          device=DEVICE)

Using device cpu.
Epoch : 0 [0/10 (0%)]	Loss: -0.541196
Epoch : 0 [2/10 (20%)]	Loss: -0.217257
Epoch : 0 [4/10 (40%)]	Loss: -0.413865
Epoch : 0 [6/10 (60%)]	Loss: -0.315132
Epoch : 0 [8/10 (80%)]	Loss: -0.387367
Epoch : 1 [0/10 (0%)]	Loss: -0.093049
Epoch : 1 [2/10 (20%)]	Loss: -0.468920
Epoch : 1 [4/10 (40%)]	Loss: -0.497781
Epoch : 1 [6/10 (60%)]	Loss: -0.499586
Epoch : 1 [8/10 (80%)]	Loss: -0.496054
Epoch : 2 [0/10 (0%)]	Loss: -0.330506
Epoch : 2 [2/10 (20%)]	Loss: -0.452206
Epoch : 2 [4/10 (40%)]	Loss: -0.508169
Epoch : 2 [6/10 (60%)]	Loss: -0.542794
Epoch : 2 [8/10 (80%)]	Loss: -0.373921
Epoch : 3 [0/10 (0%)]	Loss: -0.616050
Epoch : 3 [2/10 (20%)]	Loss: -0.543908
Epoch : 3 [4/10 (40%)]	Loss: -0.467303
Epoch : 3 [6/10 (60%)]	Loss: -0.306123
Epoch : 3 [8/10 (80%)]	Loss: -0.375272
Epoch : 4 [0/10 (0%)]	Loss: -0.516700
Epoch : 4 [2/10 (20%)]	Loss: -0.121886
Epoch : 4 [4/10 (40%)]	Loss: -0.643578
Epoch : 4 [6/10 (60%)]	Loss: -0.529382
Epoch : 4 [8/10 (80%)]	Loss: -0.582723
Epoch : 5 [0

In [33]:
preds = model.predict(X_test[:2], DEVICE)

In [ ]:
plt.imshow(preds[0][0])